In [4]:


#importing libraries
import numpy as np
import pandas as pd
#keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 
from keras.wrappers.scikit_learn import KerasClassifier




from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Input, Dropout
from keras import Sequential
from keras import regularizers
from keras.layers import Dropout
from keras.utils import to_categorical

# for combining all the hyper-parameters
import itertools 
from time import time

from sklearn.pipeline import Pipeline


In [15]:

# Grid search for the best parameter
g_eta = [0.1]
g_batchSize = [20,40]
g_hiddeLayerunit1 = [4,8]
g_momentum = [0.8,0.9]
g_afHiddenLayerunit1 = ["relu", "tanh"]

In [2]:


# reading training and testing data from the csv file
trainDf = pd.read_csv ('monks-1-test.csv',header=None).to_numpy()
testDf = pd.read_csv ('monks-1-train.csv',header=None).to_numpy()
trainDf = minmax_scale(trainDf, feature_range=(0,1), axis=0)
testDf = minmax_scale(trainDf, feature_range=(0,1), axis=0)

In [3]:

#Preparing trainDf for training
trainX = trainDf[:, 1:7]
trainY = trainDf[:, 0]
#trainX = scaler.fit_transform(trainX)
#trainY = scaler.fit_transform(trainY)

#Preparing testDf for validation
testX = testDf[:, 1:7]
testY = testDf[:, 0]

In [107]:


# # one hot encode
# train_one_hot = to_categorical(trainX,num_classes = 5)
# # invert encoding
# inverted = argmax(encoded[5:6])
# print(inverted)
# train_one_hot[5:6]

0


array([[[0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.]]], dtype=float32)

In [109]:


def create_model(lr=0.1,momentum=0.9,g_hiddeLayerunit1=4,g_hiddeLayerunit2=4,activation1="tanh",activation2="tanh",activation3="sigmoid",init_mode='uniform', g_decay=0.01):
    model = Sequential()
    model.add(Dense(g_hiddeLayerunit1, input_dim=6, kernel_initializer=init_mode, activation=activation1))
    model.add(Dropout(0.4)),
    model.add(Dense(g_hiddeLayerunit2, kernel_initializer=init_mode, activation=activation2,kernel_regularizer=regularizers.l1(0.01))) # l1 regularization parameter could also be something we can also grid search for.
    model.add(Dropout(0.4)),
    model.add(Dense(1, activation3))
    sgd = SGD(lr=lr, momentum=momentum, decay=g_decay, nesterov=False,)  # We can add decay to hyper parameter list to get optimum value. 
    model.compile(optimizer=sgd, loss='mean_squared_error',metrics=['accuracy'])
    return model

In [ ]:


start=time()
model = KerasClassifier(build_fn=create_model)

# define the grid search parameters
batch_size = [20,40]
epochs = [100]
g_hiddeLayerunit1=[4,8]
g_hiddeLayerunit2=[4,8]
g_decay = [0.01] # more estimations can be added
activation1 = ["relu", "sigmoid","softmax"]
activation2 = ["relu","sigmoid","softmax"]
activation3 = ["softmax"]
init_mode = ['uniform','glorot_uniform', 'he_normal']
lr=np.arange(0.1, 0.9, 0.1).tolist()
momentum=np.arange(0.1, 0.9, 0.1).tolist()
param_grid = dict(
    lr=lr,
    g_hiddeLayerunit1=g_hiddeLayerunit1,
    g_hiddeLayerunit2=g_hiddeLayerunit2,
    activation1=activation1,
    activation2=activation2,
    activation3=activation3,
    batch_size=batch_size,
    init_mode=init_mode,
    g_decay = g_decay,
    epochs=epochs)


grid = GridSearchCV(estimator=model, param_grid=param_grid,n_jobs=-1,cv=3,verbose=2)



grid_result = grid.fit(trainX, trainY)

end=time()

print("Total Running Time: %f",end-start)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print("Mean\tSTD\tParams")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param)) 

Fitting 3 folds for each of 3456 candidates, totalling 10368 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   45.6s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.2min
/home/anelay/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  7.4min


In [5]:
# model for monke1

def create_larger():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=6, activation='relu'))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

NameError: name 'StratifiedKFold' is not defined